# Analyse interactive
## Initialisation
Utilisons Spark pour analyser sommairement un jeu de données.

On doit dans un premier temps importer le module Python pour Spark

In [ ]:
import pyspark

On doit ensuite créer un contexte Spark. Si l'application Jupyter a été lancé à l'aide de `pyspark`, le contexte est déjà créé, et une exception de type `ValueError` est lancée. Cette exception est décélé et on affiche tout simplement un message d'avertissement.

In [ ]:
try:
    sc = pyspark.SparkContext()
except ValueError:
    print("Attenttion : Il existe déjà un SparkContext.")
    pass

Si on exécute Spark localement, la création du contexte à lancer Spark. Pour s'en convaincre, on peut visiter la console web de Spark: [http://localhost:4040](http://localhost:4040).

Si on se trouve sur une grappe de calcul, on peut déterminer le lien exécutant la cellule suivante.

In [ ]:
import socket
print("http://{hostname}:4040".format(hostname=socket.gethostname()))

## Création d'un jeu de données (RDD)

On va maintenant créer un RDD à partir d'un fichier texte contenant les données de visite de Wikipedia. Les données doivent se trouver dans le répertoire `data/pagecounts`.

In [ ]:
pagecounts = sc.textFile('data/pagecounts/*.gz')

Le contenu d'un fichier pagecounts ressemble à ceci
```
20090505-000000 af Spesiaal:Onlangse_wysigings 3 101681
20090505-000000 af Spesiaal:RecentChanges 2 2248
20090505-000000 af Suid-Afrika 1 30698
20090505-000000 af Tuisblad 14 155257 
20090505-000000 af Varkgriep 4 42236
20090505-000000 af Wikipedia 2 32796
```

Il s'agit d'un fichier tabulaire, où chaque ligne est une entrée distincte et les colonnes représentent
1. la date et l'heure d'échantillonnage
2. la langue
3. le titre de la page
4. le nombre de visionnement de la page
5. la taille de la page en octet.

On peut visualier quelques entrées en utilisant la méthode d'action `take` du RDD pour obtenir les K premiers éléments d'un jeu de données. Ici `K = 10`.

In [ ]:
pagecounts.take(10)

Puisque `take` retourne une liste, on peut itérer sur le résultat et l'afficher de manière plus lisible.

In [ ]:
for item in pagecounts.take(10):
    print(item)

### Obtenir de l'aide

À tout moment, vous pouvez obtenir de l'aide sur tout objet Python en appelant la fonction `help`. Par exemple, si on voulait en apprendre plus sur la méthode `take` du RDD:

In [ ]:
help(pagecounts.take)

## Action sur un jeu de données

La méthode `take` n'est qu'une parmi plusieurs *actions* que l'on peut effectuer sur un RDD. Une liste exhaustive des actions est disponible à l'adresse suivante:
https://spark.apache.org/docs/latest/programming-guide.html#actions

Si on ne veut pas quitter le notebook, on peut appeler directement `help` sur le RDD.

In [ ]:
help(pagecounts)

Prenons par exemple la méthode `count` qui retourne le nombre d'éléments dans un RDD.

In [ ]:
pagecounts.count()

Chaque action commise sur un RDD entraîne la création d'une ou de plusieurs tâches et la production d'un résultat. Toutes les tâches réalisées dans un même contexte Spark peuvent être visualisées dans la console web de Spark: [http://<**hostname**>:4040/](http://hostname:4040/)

À partir de cette interface, on peut suivre la progression d'une tâche, et consulter diverses mesures sur l'exécution de la tâche, par exemple sa durée et les statistiques de cache.

## Transformation d'un jeu de données

Si on reprend les 10 premiers éléments de notre jeu de données

In [ ]:
first10 = pagecounts.take(10)
first10

On constate que le RDD est formé de chacune des lignes de notre fichier d'entrées, mais qu'il nous est impossible d'accéder aux colonnes en tant que telles. **Pourquoi?**

In [ ]:
pagecounts.first()

Nous allons donc devoir transformer ce premier RDD en un second de manière à ce que chaque chaîne de caractères soit divisées en une liste de cing éléments. Pour ce faire, on utilise la fonction `string.split`. 

Importons d'abord le module string:

In [ ]:
import string

Appliquons ensuite la fonction `string.split` sur le premier élément:

In [ ]:
string.split(pagecounts.first())

On veut maintenant appliquer cette transformation à tous les éléments du RDD. La méthode `map` applique à chaque élément d'un RDD une fonction fournie en argument et retourne un nouveau RDD.

In [ ]:
pagecounts_tab = pagecounts.map(string.split)

L'évaluation d'une transformation sur un RDD est dite _lazy_ ou paresseuse. Spark n'effectue aucun travail tant que le contenu du RDD n'est pas sollicité par une action. Pour vous en convaincre, visitez la [console web Spark](http://<hostname>:4040/jobs/), appliquez un map, puis retournez à la console.

Vous constaterez qu'aucun "job" ne s'est ajouté à la liste.

## Mise en cache d'un RDD

Lorsque l'on s'attend à effectuer plusieurs opérations sur un même jeu de données, il peut être utile de spécifier à Spark de le garder en mémoire.

Pour ce faire on utilise la méthode `cache`:

In [ ]:
pagecounts_tab.cache()

Le jeu de données n'est transféré en mémoire que lorsqu'une action est appliquée. Les RDD stockés en mémoire peuvent être visualisés dans la section **Storage** de l'interface web de Spark.

Pour libérer l'espace mémoire prise par un RDD en cache dont on n'aurait plus besoin, on appelle la méthode `unpersist`.

In [ ]:
pagecounts_tab.unpersist()

## Filtrer un RDD

Comme on dispose maintenant d'un nouveau RDD plus facile à manipuler, on peut débuter l'analyse. Intéressons nous d'abord aux pages en langue anglaise.

La ligne suivante filtre le dernier RDD que nous avons créé et ne conserve que les entrées en anglais.
* Quel genre d'argument prend la fonction `filter`?
* Qu'est-ce que signifie le mot clé **`lambda`**?
* Est-ce que `filter` retourne un RDD?

In [ ]:
pagecounts_en = pagecounts_tab.filter(lambda list_: list_[1] == "en").cache()

On peut maintenant compter le nombre de pages en anglais:

In [ ]:
%time pagecounts_en.count()

Puisque qu'on a indiqué à Spark de conserver en mémoire ce nouveau jeu de données, le temps nécessaire pour effectuer le décompte du nombre de pages devrait être plus court lors de la deuxième exécution.

In [ ]:
%time pagecounts_en.count()

Comme nous avons finalement appliqué une action sur un RDD en cache, ce dernier devrait maintenant être figuré dans l'interface **Storage** de la [console Spark](http://<hostname>:4040/storage/).

## Opération de réduction

On s'intéresse maintenant à faire un diagramme à bande du nombre de pages vues dans chacune des langues de notre jeu de données. Pour ce faire, nous allons devoir procéder à une transformation de type _réduction_ de notre jeu de données.

Dans un premier temps, on transforme notre jeu de données pour conserver seulement la langue et le nombre de vues. Le nombre de vue étant une chaîne de caractères, nous utilisons la fonction `int` pour obtenir convertir la chaîne en entier.

In [ ]:
pagecounts_tuple = pagecounts_tab.map(lambda entry: (entry[1], int(entry[3])))

Visualiser les 5 premiers éléments de ce nouveau RDD pour confirmer qu'il s'agit du bon format.

In [ ]:
pagecounts_tuple.take(5)

La transformation que nous avons effectuée permet à Spark de reconnaître notre RDD comme un ensemble de paires clé-valeur. On peut donc maintenant utiliser les fonctions basées sur ce type structure. Par exemple, on peut créer un nouveau RDD contenant seulement les clés, soit les langues en appelant la méthode `keys`.

In [ ]:
pagecounts_tuple.keys().take(5)

On veut calculer le nombre total de page s ue s our chaque langue. Pour ce faire, on utilise la fonction `reduceByKey`. Cette fonction s'attend à ce que chaque entrée du RDD soit structuré comme des paires clé-valeur. Dans notre cas, la clé est la langue et la valeur est le nombre de vue. 

`reduceByKey` va combiner les valeurs pour chacune des clés en utilisant une fonction de réduction associative.
La méthode prend en argument une fonction prenant en argument deux valeurs et retournant le reéultat de leur combinaison. Dans notre cas, on effectue la somme des valeurs.

In [ ]:
lang_pagecounts = pagecounts_tuple.reduceByKey(lambda x, y: x + y)

Puisque la réduction est une transformation, le résultat est un nouveau RDD.

Pour visualiser la totalité du contenu de ce dernier RDD, on peut appeler la fonction `collect`.

In [ ]:
lang_pagecounts_local = lang_pagecounts.collect()
print(lang_pagecounts_local)

Pour obtenir seulement les 5 langues les plus populaires, il faut effectuer un tri. Plusieurs choix s'offrent à nous:

1- Effectuer le tri localement

In [ ]:
top5 = sorted(lang_pagecounts_local, key=lambda x: x[1], reverse=True)[:5]
print(top5)

2- Utiliser Spark pour effectuer un tri distribué en utilisant la fonction `sortByKey`.

In [ ]:
top5 = lang_pagecounts.map(lambda x: (x[1], x[0]))\
                      .sortByKey(False)\
                      .map(lambda x: (x[1], x[0]))\
                      .take(5)
print(top5)

3- Utiliser la méthode `top` du RDD:

In [ ]:
top5 = lang_pagecounts.top(5, lambda x: x[1])
print(top5)

On peut ensuite créer un diagramme à bandes des 5 langues les plus populaires de notre jeu de données.

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
fig = plt.figure()
ax = plt.subplot(111)
top5_t = zip(*top5)
ax.bar(range(len(top5_t[0])), top5_t[1], width=0.35, align="center")
ax.set_xticks(range(len(top5_t[0])))
ax.set_xticklabels(top5_t[0])
ax.set_xlabel("langue")
ax.set_ylabel(u"décompte")

## Terminer l'analyse

Une fois l'analyse terminée, on doit mettre fin au contexte Spark à l'aide de la méthode `stop`.

In [ ]:
sc.stop()